In [1]:
import geopandas as gpd

## Open GIS Data

In [2]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_gdf = gpd.read_file(file_path)
states_gdf = states_gdf[["STUSPS", "geometry"]]

## Open Popeyes Data

In [3]:
popeyes_df = gpd.read_file("data/popeyes_per_state.gpkg")
popeyes_df["popeyes_per_100k"] = popeyes_df["per_100k"]
popeyes_df = popeyes_df[["STUSPS", "POPEYES", "popeyes_per_100k"]]

## Open KFC Data

In [4]:
kfc_gdf = gpd.read_file("data/kfcs_per_states.gpkg")
kfc_gdf["KFCs"] = kfc_gdf["stores"]
kfc_gdf["KFCs_per_100k"] = kfc_gdf["per_100k"]
kfc_gdf = kfc_gdf[["STUSPS", "KFCs", "KFCs_per_100k"]]

## Open Chick-Fil-A Data

In [6]:
cfa_gdf = gpd.read_file("data/chick_fil_a_per_state.gpkg")
cfa_gdf["cfas_per_100k"] = cfa_gdf["per_100k"]
cfa_gdf = cfa_gdf[["STUSPS", "CFAs", "cfas_per_100k"]]

## Merge Data

In [9]:
popeyes_states_gdf = states_gdf.merge(popeyes_df, on="STUSPS", how="left")
popeyes_kfc_states_gdf = popeyes_states_gdf.merge(kfc_gdf, on="STUSPS", how="left")
popeyes_kfc_cfa_states_gdf = popeyes_kfc_states_gdf.merge(
    cfa_gdf, on="STUSPS", how="left"
)

In [10]:
popeyes_kfc_cfa_states_gdf = popeyes_kfc_cfa_states_gdf.fillna(0)

In [11]:
popeyes_kfc_cfa_states_gdf["POPEYES"] = popeyes_kfc_cfa_states_gdf["POPEYES"].astype(
    int
)
popeyes_kfc_cfa_states_gdf["KFCs"] = popeyes_kfc_cfa_states_gdf["KFCs"].astype(int)
popeyes_kfc_cfa_states_gdf["CFAs"] = popeyes_kfc_cfa_states_gdf["CFAs"].astype(int)

In [26]:
def more_stores(row) -> str:
    popeyes = row["POPEYES"]
    kfc = row["KFCs"]
    cfa = row["CFAs"]
    if (popeyes > kfc) and (popeyes > cfa):
        return "More Popeyes"
    if (kfc > popeyes) and (kfc > cfa):
        return "More KFC"
    if (cfa > popeyes) and (cfa > kfc):
        return "More Chick-Fil-A"
    if popeyes == 0 and kfc == 0 and cfa == 0:
        return "No Locations"
    if popeyes == kfc:
        return "Same Amount of Popeyes And KFCs"
    else:
        return "Edge Case"

In [27]:
popeyes_kfc_cfa_states_gdf["label"] = popeyes_kfc_cfa_states_gdf.apply(
    lambda row: more_stores(row), axis=1
)

In [28]:
popeyes_kfc_cfa_states_gdf[popeyes_kfc_cfa_states_gdf["label"] == "Edge Case"]

,STUSPS,geometry,POPEYES,popeyes_per_100k,KFCs,KFCs_per_100k,CFAs,cfas_per_100k,label


In [29]:
popeyes_kfc_cfa_states_gdf = popeyes_kfc_cfa_states_gdf.to_crs(9311)
popeyes_kfc_cfa_states_gdf.to_file("data/popeyes_kfc_cfas.gpkg")